In [279]:
import pandas as pd
from pyspark.sql import SparkSession
from datetime import date

In [281]:
def format_date(date):
    temp = date.split("/")
    temp.insert(0, temp[2])
    temp = list(map(int,temp[:3]))
    return temp

big_list = []
with open('YelpData.txt', 'rb') as f:
    data = f.readlines()
for partial_data in data:
    df = pd.read_json(partial_data)
    for column in df.columns:
        temp = list(df[column])
        temp.insert(0,column)
        big_list.append(temp)
new_df = pd.DataFrame(big_list,columns=["key", "website", "restaurant","date", "review", "rating", "num_votes"])
new_df.date = new_df.date.map(lambda x: date(*format_date(x)))

In [282]:
def rating_map(rest_rating, rating):
    if rating == rest_rating:
        return 1
    return 0

def get_ratings(df): # takes in pandas DataFrame
    rating = df[["restaurant", "rating"]]
    for i in range(1,6):
        rating["rating_" + str(i)] = rating.rating.apply(lambda x: rating_map(i,x))
    rating = rating.groupby("restaurant").sum()
    return rating

In [283]:
get_ratings(new_df).head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,rating,rating_1,rating_2,rating_3,rating_4,rating_5
restaurant,,,,,,
Arby’s,7,0,0,1,1,0
Calios,21,2,0,1,4,0
Chili’s,51,4,2,4,4,3
Denny’s,80,16,8,2,3,6
Friendly’s,46,7,2,2,1,5


In [235]:
spark = SparkSession.builder.appName('restaurant_reviews').getOrCreate()
spark_df = spark.createDataFrame(new_df)
# spark_df.write.saveAsTable("yelp")

In [236]:
a = spark_df.groupBy("restaurant").count()

In [237]:
a.show()

+--------------------+-----+
|          restaurant|count|
+--------------------+-----+
|        Gorgers Subs|  140|
|            Tamarind|  106|
|     Napoli Pizzeria|   48|
|Luna Inspired Str...|   90|
|Applebee’s Grill ...|   27|
|  Buffalo Wild Wings|   31|
|              Subway|    1|
|       Stella’s Barn|   27|
|Capital State Kit...|   20|
|         Sushi Osaka|    9|
|     Gangnam Station|   31|
|Spring Buffet & G...|   37|
| Dapper’s Restaurant|    6|
|Taste of Thai Exp...|  166|
|           Taco Bell|   16|
|         Pizza Aroma|   55|
|       Taste of Thai|  180|
|         Atrium Cafe|    3|
|Little Caesars Pizza|    3|
|         Nikki Green|   33|
+--------------------+-----+
only showing top 20 rows

